In [1]:
# Preprocessing data for use in developing a model to predict water usage above the daily allowed rates
# for given neighbourhoods.
# Note accompanying data (and their consequent representation in this process) were changed (agreement terms)
# The code was also changed, only ensuring appropriate 'Key areas' are maintained to allow the executions of
# which showcase my data science skills using python. 
# The whole process is re-usable for other cases and data, with necessary twitch of some parameters in the code, 

# Importing necessary modules

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
#from sklearn import metrics

In [3]:
raw_csv_data = pd.read_csv('Water_Usage_Data.csv') ## loading raw data

In [4]:
raw_csv_data.head() ## eye-balling the raw data

,ID,Date,Zone,Lat,Long,Meter Number,Reason for Over Abstraction,HouseHolds,Type of Housing,Main Pipe Length,Daily Abstraction Avg,Water Kiosks,Car Wash and Hotels,Schools,Churches,Farming,Over Abstraction
0,6,7/7/2015,Lower Kayole,0.745297,36.461339,WT11-6''-06-12840,Other_Acts,179,single units rentals,1604,134.25,2,0,1,2,1,4
1,1,15/07/2015,park view,0.744511,36.462356,WT-4''-96-4997,Mt_Pb,69,single family units,3480,51.75,2,1,0,2,3,2
2,4,16/07/2015,Upper Kayole,0.748353,36.466414,WT11-06''-12842,LK_2,169,mixed rental storied (2 bedroomed plus) and fa...,1750,126.75,0,0,0,0,3,4
3,6,23/07/2015,Lower Kayole,0.745297,36.461339,WT11-6''-06-12840,Mt_Pb,179,single units rentals,1604,134.25,2,0,1,2,1,2
4,1,10/7/2015,park view,0.744511,36.462356,WT-4''-96-4997,Mt_Pb,69,single family units,3480,51.75,2,1,0,2,3,2


In [5]:
df = raw_csv_data.copy() ##making a copy of data to work with to avoid changing the raw data

In [6]:
df = df.drop(['ID', 'Lat','Long','Meter Number', 'Zone'], axis=1) ##dropping columns not needed for the analysis

# Dealing with the Categorical  Variables/ Columns

# Reasons for over abstraction

In [7]:
df['Reason for Over Abstraction'].unique()

array(['Other_Acts', 'Mt_Pb', 'LK_2', 'New_Cons', 'Htl_Inds', 'LK_1',
       'LK_3', 'Rare_Acts', 'Insts', 'Illegal_Abs', '0'], dtype=object)

getting dummies for the reasons column

In [8]:
reason_columns = pd.get_dummies(df['Reason for Over Abstraction'], drop_first=True)

In [9]:
reason_columns['check'] = reason_columns.sum(axis = 1) ## creating a column to check the values in the reason_columns

In [10]:
reason_columns['check'].unique()

array([1, 0], dtype=int64)

In [11]:
reason_columns = reason_columns.drop(['check'], axis = 1) ## dropping the created column

Classification of over abstraction reasons 

In [12]:
##Mapping different types of over abstraction to the decided classes
Leakage = reason_columns.loc[:,'LK_1':'LK_3'].max(axis=1)
Over_Demand = reason_columns.loc[:,'Htl_Inds':'Insts'].max(axis=1)
Unique_Reasons = reason_columns.loc[:,'Mt_Pb':'Rare_Acts'].max(axis=1)

In [13]:
df=pd.concat([df,Leakage,Over_Demand,Unique_Reasons], axis=1) ##concatenating the classified reasons to the dataframe

In [14]:
df = df.drop(['Reason for Over Abstraction'], axis=1) ##Dropping the "Reason for Over Abstraction" column

In [15]:
df.columns.values ##confirming the new column values of the dataframe

array(['Date', 'HouseHolds', 'Type of Housing', 'Main Pipe Length',
       'Daily Abstraction Avg', 'Water Kiosks', 'Car Wash and Hotels',
       'Schools', 'Churches', 'Farming', 'Over Abstraction', 0, 1, 2],
      dtype=object)

In [16]:
##creating a list of names to update the classified reasons for over abstraction column names 
column_names = ['Date', 'HouseHolds', 'Type of Housing', 'Main Pipe Length',
       'Daily Abstraction Avg','Water Kiosks',
       'Car Wash and Hotels', 'Schools', 'Churches', 'light irrigation',
       'Over Abstraction', 'Leakage','Over_Demand','Unique_Reasons'] 

In [17]:
df.columns=column_names

Obtaining Dummies for the "type of housing" column

In [18]:
Housing_Types_columns = pd.get_dummies(df['Type of Housing']) ##dummy variables for the housing types


In [19]:
##Re-ordering housing types for easier classification using the loc method
columns_re_ordered = ['single family units','mixed family and single rentals',
                      'mixed rentals and homes','mixed rental storied (2 bedroomed plus) and family units',
                      'one bedroomed plus','rentals storied (2 bedroomed plus)',
                      'Single Units rentals','single units rentals']

In [20]:
Housing_Types_columns = Housing_Types_columns[columns_re_ordered]

In [21]:
##classifying housing types to three levels of population "lowpop, midpop, and overpop"
OverPop = Housing_Types_columns.loc[:,'one bedroomed plus':'single units rentals'].max(axis=1)
MidPop = Housing_Types_columns.loc[:,'mixed family and single rentals':'mixed rental storied (2 bedroomed plus) and family units'].max(axis=1)
LowPop = Housing_Types_columns.loc[:,'single family units']

In [22]:
df=pd.concat([df,OverPop,MidPop,LowPop], axis=1) ##concatenating the newly obtained population classes to the df

In [23]:
## renaming the population classes columns and assigning this to the df 
columns_names_2 = ['Date', 'HouseHolds', 'Type of Housing', 'Main Pipe Length',
       'Daily Abstraction Avg', 'Water Kiosks',
       'Car Wash and Hotels', 'Schools', 'Churches', 'Farming',
       'Over Abstraction', 'Leakage', 'Over_Demand', 'Unique_Reasons', 'OverPop' ,
       'MidPop', 'LowPop']

In [24]:
df.columns=columns_names_2

In [25]:
df = df.drop(['Type of Housing'], axis=1) ##dropping the "Type of housing column as it is now represented in the new classes"

In [26]:
## reordering the column names so the reasons for over-abstractions can be at the start of the df 
column_names_reodered = ['Leakage',
       'Over_Demand', 'Unique_Reasons','Date', 'HouseHolds', 'Main Pipe Length', 'Daily Abstraction Avg',
       'Water Kiosks', 'Car Wash and Hotels', 'Schools',
       'Churches', 'Farming', 'Over Abstraction', 'OverPop', 'MidPop', 'LowPop']

In [27]:
df = df[column_names_reodered]
#df.head()

In [28]:
## checking the columns "Schools", 'Churches', 'Water Kiosks' to determine if they need any pre-processing

In [29]:
df['Schools'].unique()

array([1, 0], dtype=int64)

In [30]:
df['Churches'].unique()

array([2, 0, 3, 5, 1], dtype=int64)

In [31]:
df['Water Kiosks'].unique() ## important for analysis as indicates water usage by unconnected homes and water trucks

array([2, 0, 1], dtype=int64)

In [32]:
df['Farming'].value_counts() #analysing the "Farming column"

4    246
1    196
3    126
2     73
Name: Farming, dtype: int64

In [33]:
## The farming column data is categorical, with farming types classified into 3 levels based on water use
##level 1,2,3 ("poultry,pigs and goats", "kitchen gardening", "Dairy Cows", respectively); use is negligible, 
##but 4 (light irrigation) is significant.

In [34]:
## classifying the column into "Significant impact = 1" and "Less impact = 0" using dictionary
df['Farming'] = df['Farming'].map({1:0, 2:0, 3:0, 4:1})

In [35]:
df['Farming'].value_counts() ##confirming the classification of the column

0    395
1    246
Name: Farming, dtype: int64

# Date Column

In [36]:
## Note: it is important to assess on if the over abstractions 
#can be associated to certain months and also certain days of the week.
##Thus spliting the date information to obtain months values and days of the week

In [37]:
type(df['Date'][0]) # Date is in string data type, thus need to change to time stamp

str

In [38]:
df['Date'] =pd.to_datetime(df['Date'],format = '%d/%m/%Y') #changing to Pandas timestamps
df['Date']

0     2015-07-07
1     2015-07-15
2     2015-07-16
3     2015-07-23
4     2015-07-10
         ...    
636   2018-05-23
637   2018-05-23
638   2018-05-24
639   2018-05-24
640   2018-05-31
Name: Date, Length: 641, dtype: datetime64[ns]

In [39]:
type(df['Date'][0]) ##confirming date data type conversion

pandas._libs.tslibs.timestamps.Timestamp

In [40]:
df['Date'][0].month #checking the month information

7

In [41]:
list_months = [] ##creating an empty list to hold the month values

In [42]:
#iterating throught the data frame for the date column to obtaing the month values and appending them to the list
for i in range(df.shape[0]):
    list_months.append(df['Date'][i].month)

In [43]:
df['Month Value'] = list_months #inserting the Month Value information to the df

In [44]:
# function to obtain the day of the week, using the "date_value.weekday"
def date_to_weekday(date_value):
    return date_value.weekday()

In [45]:
df['Day of the Week'] = df['Date'].apply(date_to_weekday) #calling the function and applying it to the df

In [46]:
df = df.drop(['Date'], axis=1) #droping the "Date columns": the important information for analysis
                                # is now contained in the "Month value" and "Day of the week" columns

In [47]:
df.head() #confirming the date columns operations

,Leakage,Over_Demand,Unique_Reasons,HouseHolds,Main Pipe Length,Daily Abstraction Avg,Water Kiosks,Car Wash and Hotels,Schools,Churches,Farming,Over Abstraction,OverPop,MidPop,LowPop,Month Value,Day of the Week
0,0,0,1,179,1604,134.25,2,0,1,2,0,4,1,0,0,7,1
1,0,0,1,69,3480,51.75,2,1,0,2,0,2,0,0,1,7,2
2,1,0,0,169,1750,126.75,0,0,0,0,0,4,0,1,0,7,3
3,0,0,1,179,1604,134.25,2,0,1,2,0,2,1,0,0,7,3
4,0,0,1,69,3480,51.75,2,1,0,2,0,2,0,0,1,7,4


In [61]:
df.columns.values #obtaining columns values for reordering for the date information to be at the start of the df

array(['Leakage', 'Over_Demand', 'Unique_Reasons', 'Month Value',
       'Day of the Week', 'HouseHolds', 'Main Pipe Length',
       'Daily Abstraction Avg', 'Water Kiosks', 'Car Wash and Hotels',
       'Schools', 'Churches', 'Farming', 'Over Abstraction', 'OverPop',
       'MidPop', 'LowPop'], dtype=object)

In [62]:
#Actual reordering of the columns
columns_date_reordered = ['Leakage', 'Over_Demand', 'Unique_Reasons', 'Month Value',
       'Day of the Week', 'HouseHolds',
       'Main Pipe Length', 'Daily Abstraction Avg', 'Water Kiosks',
       'Car Wash and Hotels', 'Schools', 'Churches', 'Farming',
       'Over Abstraction', 'OverPop', 'MidPop', 'LowPop']

In [63]:
df = df[columns_date_reordered] #effecting the reordering to the df

# Check Point Preprocessed data 

In [64]:
data_preprocessed = df.copy() 

In [65]:
#activate this code to output the pre-processed data
#data_preprocessed.to_csv('Water_abstraction_Preprocessed.csv', index=False)

# Creating the Model for predicting over abstraction of water

# Create the targets : "Moderate use" and "Excess use"

In [66]:
data_preprocessed['Over Abstraction'].median() #below median= "Moderate use", above = "Over use"

3.0

In [67]:
targets = np.where(data_preprocessed['Over Abstraction']>
                   data_preprocessed['Over Abstraction'].median(),1,0)

In [68]:
# create a Series in the original data frame that will contain the targets for the regression
data_preprocessed['Excess Usage'] = targets

In [69]:
data_preprocessed.head() # to see what happened

,Leakage,Over_Demand,Unique_Reasons,Month Value,Day of the Week,HouseHolds,Main Pipe Length,Daily Abstraction Avg,Water Kiosks,Car Wash and Hotels,Schools,Churches,Farming,Over Abstraction,OverPop,MidPop,LowPop,Excess Usage
0,0,0,1,7,1,179,1604,134.25,2,0,1,2,0,4,1,0,0,1
1,0,0,1,7,2,69,3480,51.75,2,1,0,2,0,2,0,0,1,0
2,1,0,0,7,3,169,1750,126.75,0,0,0,0,0,4,0,1,0,1
3,0,0,1,7,3,179,1604,134.25,2,0,1,2,0,2,1,0,0,0
4,0,0,1,7,4,69,3480,51.75,2,1,0,2,0,2,0,0,1,0


In [70]:
#check if dataset is balanced (what % of targets are 1s)
targets.sum() / targets.shape[0] 

0.46021840873634945

In [71]:
# create a checkpoint by dropping the unnecessary variables
# also drop the variables 'eliminated' after exploring the weights
data_with_targets = data_preprocessed.drop(['Over Abstraction','Daily Abstraction Avg','Main Pipe Length','OverPop'], axis = 1) # droping the Over Abstraction clmn

In [72]:
data_with_targets is data_preprocessed  ## confirming the created data frame

False

# Selecting the regression inputs

In [73]:
data_with_targets.shape

(641, 14)

In [74]:
# Selects all rows and all columns but the last one and put in a variable for inputs
unscaled_inputs = data_with_targets.iloc[ :,:-1]

# Standardizing the data

In [75]:
# since data of different magnitude (scale) can be biased towards high values,
# we want all inputs to be of similar magnitude

Custom scaler class

In [76]:
class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features chosen when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # using the original order (recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [77]:
# check what are all columns that we've got
unscaled_inputs.columns.values

array(['Leakage', 'Over_Demand', 'Unique_Reasons', 'Month Value',
       'Day of the Week', 'HouseHolds', 'Water Kiosks',
       'Car Wash and Hotels', 'Schools', 'Churches', 'Farming', 'MidPop',
       'LowPop'], dtype=object)

In [ ]:
# select the columns to omit
columns_to_omit = ['Leakage', 'Over_Demand', 'Unique_Reasons''Farming', 'MidPop',
       'LowPop', 'OverPop']

In [ ]:
# creating the columns to scale, based on the columns to omit
# using list comprehension to iterate over the list
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [ ]:
#creating a scale as an object specifying the columns to be scaled
over_demand_scaler = CustomScaler(columns_to_scale)

In [ ]:
over_demand_scaler.fit(unscaled_inputs) #calculating and storing standardization info
                                            #(mean and std)

In [ ]:
scaled_inputs = over_demand_scaler.transform(unscaled_inputs) #actual standardization (transforming)

# Data split and shuffling (train and test)

In [ ]:
# declare 4 variables for the split
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)

In [ ]:
# check the shape of the train inputs and targets
print(x_train.shape, y_train.shape)

In [ ]:
# check the shape of the test inputs and targets
print(x_test.shape, y_test.shape)

# Model Training

Use made of the Logistic regression with sklearn

In [ ]:
# create a logistic regression object
reg = LogisticRegression()

In [ ]:
# fit our train inputs i.e the training part of the machine learning
reg.fit(x_train,y_train)

In [ ]:
# assess the train accuracy of the model
reg.score(x_train,y_train)

# Finding the Intercept and coefficients

In [ ]:
# get the intercept (bias) of the model
reg.intercept_

In [ ]:
# get the coefficients (weights) of the model
reg.coef_

In [ ]:
# saving the columns names in an ad-hoc variable
feature_name = unscaled_inputs.columns.values # to restore features lost in SkLearn operation 

In [ ]:
# add the coefficient values to the summary table
summary_table = pd.DataFrame(columns=['Feature name'], data = feature_name)
summary_table['Coefficient'] = Fsk(reg.coef_)
# display the summary table
summary_table

To add the intercept to the summary table

In [ ]:
# to move the intercept to the top of the summary table
summary_table.index = summary_table.index + 1 #shift up all indices by 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]] #Add the Intercept index
summary_table = summary_table.sort_index() #sort the df by index to organize the indices
summary_table

# Interpreting coefficients

In [ ]:
# create a new Series called: 'Odds ratio' which will show the.. odds ratio of each feature
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)
summary_table

In [ ]:
# sort the table according to odds ratio
# NB: by default, the sort_values method sorts values by 'ascending'
summary_table.sort_values('Odds_ratio', ascending=False)

In [ ]:
## Note if coefficient is close to zero, or odds ratio close to 1, 
## the feature is less important

# Model Test

In [ ]:
# running the model with the test data to test its performance
reg.score(x_test,y_test)

In [ ]:
# obtaining the probabilities of both Moderate and Over abstraction
predicted_proba = reg.predict_proba(x_test)
predicted_proba

In [ ]:
# Obtaining the predictied probabilies of excess abstraction
predicted_proba[ :,1]